In [1]:
import numpy as np
import readline
%pylab inline
%load_ext autoreload
%autoreload 2
%precision 3

Populating the interactive namespace from numpy and matplotlib


'%.3f'

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
lemma = nltk.wordnet.WordNetLemmatizer()

tt = TweetTokenizer(preserve_case=False)

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
stops.update({"#semst", "...", "rt"})
ps = nltk.stem.porter.PorterStemmer()

def tokenize(t):
    tokens = tt.tokenize(t)
    tokens = [tok for tok in tokens if not ((len(tok) == 1 and not tok.isalpha()) or tok in stops)]
    tokens = [ (ps.stem(tok) if tok[0] != "#" else tok) for tok in tokens]
    return tokens

In [3]:
train = pd.read_csv("../semeval2016.6/StanceDataset/train.csv", encoding= 'unicode_escape', engine='python')
del train["Opinion Towards"]
del train["Sentiment"]
train["tokens"] = [ tokenize(t) for t in train["Tweet"].values ]
train["num_tokens"] = [len(tt) for tt in train["tokens"].values ]
train.head()

,Tweet,Target,Stance,tokens,num_tokens
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,"[@tedcruz, #handovertheserver, wipe, clean, 30...",14
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,"[hillari, best, choic, truli, want, continu, p...",9
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,"[@theview, think, countri, readi, femal, pre, ...",9
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,"[gave, unhealthi, amount, hard-earn, money, aw...",11
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,"[@portiaaboulg, thank, ad, list]",4


In [4]:
test = pd.read_csv("../semeval2016.6/StanceDataset/test.csv", encoding= 'unicode_escape', engine='python')
del test["Opinion Towards"]
del test["Sentiment"]
test["tokens"] = [ tokenize(t) for t in test["Tweet"].values ]
test["num_tokens"] = [len(tt) for tt in test["tokens"].values ]
test.head()

,Tweet,Target,Stance,tokens,num_tokens
0,He who exalts himself shall be humbled; a...,Atheism,AGAINST,"[exalt, shall, humbl, humbl, shall, exalted.ma...",7
1,RT @prayerbullets: I remove Nehushtan -previou...,Atheism,AGAINST,"[@prayerbullet, remov, nehushtan, previou, mov...",12
2,@Brainman365 @heidtjj @BenjaminLives I have so...,Atheism,AGAINST,"[@brainman365, @heidtjj, @benjaminl, sought, t...",11
3,#God is utterly powerless without Human interv...,Atheism,AGAINST,"[#god, utterli, powerless, without, human, int...",6
4,@David_Cameron Miracles of #Multiculturalism...,Atheism,AGAINST,"[@david_cameron, miracl, #multiculturalism, mi...",13


In [5]:
pd.crosstab(index=train['Target'], columns=train['Stance'], margins = True)

Stance,AGAINST,FAVOR,NONE,All
Target,,,,
Atheism,304,92,117,513
Climate Change is a Real Concern,15,212,168,395
Feminist Movement,328,210,126,664
Hillary Clinton,393,118,178,689
Legalization of Abortion,355,121,177,653
All,1395,753,766,2914


In [6]:
def search(target, keyword):
    subset = train.query('Target==@target')
    filtered = subset[subset.apply(lambda row: keyword in row["tokens"], axis=1)]
    return filtered[["Tweet", "Stance"]]#.reset_index()

In [7]:
from collections import Counter
from itertools import chain, product
all_tokens = Counter(chain(*train.query('Target=="Atheism"')["tokens"]))
# plt.hist(np.log2(list(all_tokens.values()) ), log=True)
all_tokens.most_common(20)

[('god', 101),
 ('#god', 50),
 ('love', 35),
 ('us', 34),
 ('religion', 33),
 ('peopl', 31),
 ('faith', 29),
 ('believ', 26),
 ('jesu', 26),
 ('lord', 25),
 ('day', 25),
 ('one', 25),
 ('go', 25),
 ('life', 24),
 ('make', 23),
 ('know', 23),
 ('world', 20),
 ('#islam', 20),
 ('#lovewins', 18),
 ('like', 17)]

In [14]:
pd.options.display.max_colwidth = 150
# AGAINST: jesu, lord, bless | FAVOR: #freethinker
# AGAINST #gamergate #spankafeminist 
# AGAINST #prolifeyouth, kill, murder, life
# AGAINST #wakeupamerica email #benghazi | @hillaryclinton @barackobama support
# FAVOR #environment
# print(search("Hillary Clinton", "right").to_string())

In [9]:
def assign(target, pos_kws, neg_kws):
    subset = train.query('Target==@target')
    pos_idxs = subset.apply(lambda row: len (set(pos_kws) & set(row["tokens"])) > 0, axis=1).values
    neg_idxs = subset.apply(lambda row: len (set(neg_kws) & set(row["tokens"])) > 0, axis=1).values
    inferred = -1.0 * np.ones(len(subset))
    inferred[pos_idxs] = 1.0
    inferred[neg_idxs] = 0.0
    subset["labels"] = inferred
    return subset

In [10]:
from submodular.models2 import *
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer

In [11]:
lst = ['AGAINST', 'FAVOR']

HYPERPARAMS_GRID = {
    "lambdaa": [0.1, 0.2, 0.3, 0.4],
    "C": [0.1, 1.0, 10.0, 100.0],
    "gamma": [0.0625, 0.125, 0.25, 0.5, 1.0, 2.0],
    "alpha": [0.8, 0.85, 0.9]
}

In [23]:
from commons.functions import avg_f1 as scorer_func
from commons.functions import each_f1
mks = [5]
CV = 3
N_JOBS = 6 #int(sys.argv[1])
scorer = make_scorer(scorer_func)
STEPS = [
    # ["tok", "idf-bow", "cos", "greedy-diff"],
    ["tok", "idf-bow", "exp", "greedy-diff"],
    # ["tok", "idf-bow", "cos", "greedy-div"],
    ["tok", "idf-bow", "exp", "greedy-div"],
    # ["tok", "bow", "cos", "greedy-diff"],
    ["tok", "bow", "exp", "greedy-diff"],
    # ["tok", "bow", "cos", "greedy-div"],
    ["tok", "bow", "exp", "greedy-div"],
    ["tok", "idf-bow", "length"],
    # ["tok", "idf-bow", "cos", "random"],
    ["tok", "idf-bow", "cos", "pr"],
    ["tok", "bow", "cos", "pr"],
    ["tok", "idf-bow", "cos", "nn"],
    ["tok", "bow", "cos", "nn"],
    ["tok", "bow", "cos", "all"],
    ["tok", "idf-bow", "cos", "all"],
    ["tok", "bow", "exp", "all"],
    ["tok", "idf-bow", "exp", "all"]
]

In [24]:
target, pos, neg = "Hillary Clinton", ["#lovewins", "@barackobama", "women", "proud"], ["#wakeupamerica", "email", "#benghazi"]
# target, pos, neg = "Atheism", ["#freethinker", "#freedom", "religi"], ["jesu", "lord", "bless", "#teamjesus"]
subset_train = assign(target, pos, neg)
pd.crosstab(index=subset_train['labels'], columns=subset_train['Stance'], margins = True)

/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Stance,AGAINST,FAVOR,NONE,All
labels,,,,
-1.0,325,97,160,582
0.0,65,1,7,73
1.0,3,20,11,34
All,393,118,178,689


In [25]:
subset_test = test.query('Target==@target')
test_target = test.query('Target == @target & Stance == @lst').reset_index()
train_target = subset_train.query('Target == @target & Stance == @lst & labels != -1').reset_index()

y_tr = np.array( 2 * train_target.labels - 1, dtype = int)
y_te = np.ones(len(test_target), dtype = np.int8)

# y_tr[np.where(train_target.Stance.values == "AGAINST")] = -1
y_te[np.where(test_target.Stance.values == "AGAINST")] = -1
train_target["y"] = y_tr
test_target["y"] = y_te
    
print(Counter(y_tr), Counter(y_te))


for steps, mk in product(STEPS, mks):
    pipe, _hyperparams = get_model(steps, mk = mk)
    name = "_".join(steps)
    hyperparams = {}
    for hp in _hyperparams:
        hp_name, step = hp
        hyperparams["{}__{}".format(step, hp_name)] = HYPERPARAMS_GRID[hp_name]
        # if step == "greedy-div" and hp_name == "lambdaa":
        #     hyperparams["{}__{}".format(step, hp_name)] = HYPERPARAMS_GRID["lambda2"]
#     print("{}, m={}, params={}".format(name, mk, ";".join(hyperparams.keys()) ))

    clf = GridSearchCV(Pipeline(pipe), hyperparams, 
                cv = CV, n_jobs = N_JOBS, pre_dispatch = N_JOBS, verbose = 0, scoring = scorer, 
                return_train_score = False, error_score = 0.5)
    clf.fit(train_target, train_target.y.values)
    train_score, test_score = clf.score(train_target, train_target.y.values), clf.score(test_target, test_target.y.values)
    val_score = clf.best_score_
    each_f1_scores = each_f1(y_te, clf.predict(test_target) )
#     print("{} => {}[{}]: val:{:.4g}, test:{:.4g} | each_f1:{}".format(target, name, mk, val_score, test_score, each_f1_scores.tolist()))
    S = clf.best_estimator_.steps[-1][1].idxs
    Summaries = train_target.iloc[S].Tweet.values
    Summaries1 = Summaries[:mk].tolist()
    Summaries2 = Summaries[mk:].tolist()
    summ_lengths = [np.mean(train_target.iloc[S].num_tokens.values[:mk]), np.mean(train_target.iloc[S].num_tokens.values[mk:])]
    msg = "{},{},{},{:.4g},{:.4g},{:.2g},{:.2g},{:.4g},{:.4g},{:.4g}".format(
        target, name, mk, val_score, test_score,
        *summ_lengths,
        *each_f1_scores,
        np.mean(each_f1_scores)
        # " ###### ".join(Summaries1),
        # " ###### ".join(Summaries2)
    )
    print(clf.best_estimator_)
    print(msg)
    print()

Counter({-1: 66, 1: 23}) Counter({-1: 172, 1: 45})


/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Jul-22 07:19:17 INFO [MMDGreedy:275]=> fitted MMDGreedy(C=0.1,lambda=0.1,m=5)


Pipeline(steps=[('tok', PandasSelector(tokens)),
                ('idf-bow', BoW(weights='idf')), ('exp', ExpK(gamma=0.0625)),
                ('greedy-diff', MMDGreedy(C=0.1, lambdaa=0.1, mk=5))])
Hillary Clinton,tok_idf-bow_exp_greedy-diff,5,0.8495,0.543,8.8,8.6,0.7157,0.3704,0.543



/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Jul-22 07:19:18 INFO [MMDGreedy:275]=> fitted MMDGreedy(C=0.1,lambda=0.1,m=5)


Pipeline(steps=[('tok', PandasSelector(tokens)),
                ('idf-bow', BoW(weights='idf')), ('exp', ExpK(gamma=0.0625)),
                ('greedy-div',
                 MMDGreedy(C=0.1, diff='div', lambdaa=0.1, mk=5))])
Hillary Clinton,tok_idf-bow_exp_greedy-div,5,0.8384,0.5597,8.8,8.4,0.7829,0.3364,0.5597



...

Pipeline(steps=[('tok', PandasSelector(tokens)), ('bow', BoW()),
                ('cos', CosineK()), ('all', AllComp(C=0.1))])
Hillary Clinton,tok_bow_cos_all,5,0.5,0.5347,12,9.9,0.8842,0.1852,0.5347

Pipeline(steps=[('tok', PandasSelector(tokens)),
                ('idf-bow', BoW(weights='idf')), ('cos', CosineK()),
                ('all', AllComp(C=0.1))])
Hillary Clinton,tok_idf-bow_cos_all,5,0.5,0.465,12,9.9,0.8866,0.04348,0.465



/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Jul-22 07:19:21 INFO [AllComp:275]=> fitted AllComp(C=0.1,m=all)


Pipeline(steps=[('tok', PandasSelector(tokens)), ('bow', BoW()),
                ('exp', ExpK(gamma=0.0625)), ('all', AllComp(C=0.1))])
Hillary Clinton,tok_bow_exp_all,5,0.5,0.6074,12,9.9,0.7938,0.4211,0.6074



/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Jul-22 07:19:22 INFO [AllComp:275]=> fitted AllComp(C=0.1,m=all)


Pipeline(steps=[('tok', PandasSelector(tokens)),
                ('idf-bow', BoW(weights='idf')), ('exp', ExpK(gamma=0.0625)),
                ('all', AllComp(C=0.1))])
Hillary Clinton,tok_idf-bow_exp_all,5,0.5,0.5538,12,9.9,0.854,0.2535,0.5538



In [26]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_kernels as kernels

In [34]:
bow = TfidfVectorizer(
            tokenizer = lambda x: x, ## we already have list of entities
            lowercase = False,
            binary = False,
            use_idf = True,
            preprocessor = None,
            norm = None,
            min_df = 2,
        )
X = bow.fit_transform(subset_train["tokens"])
model = LabelPropagation(kernel = lambda X, Y: kernels(X, Y, metric = "cosine"))
model.fit(X.todense(), subset_train["labels"].values)
subset_train["predict"] = model.transduction_
pd.crosstab(index=subset_train['predict'], columns=subset_train['Stance'], margins = True)

/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/semi_supervised/_label_propagation.py:403: RuntimeWarning: invalid value encountered in true_divide
  affinity_matrix /= normalizer[:, np.newaxis]
/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/semi_supervised/_label_propagation.py:281: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/semi_supervised/_label_propagation.py:292: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning
/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

Stance,AGAINST,FAVOR,NONE,All
predict,,,,
0.0,390,98,167,655
1.0,3,20,11,34
All,393,118,178,689


In [35]:
subset_test1 = test.query('Target=="Atheism"')
X_test = bow.transform(subset_test1["tokens"])
subset_test1['predict'] = model.predict(X_test)
pd.crosstab(index=subset_test1['predict'], columns=subset_test1['Stance'], margins = True)

/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Stance,AGAINST,FAVOR,NONE,All
predict,,,,
0.0,160,32,28,220
All,160,32,28,220


In [36]:
# target, pos, neg = "Hillary Clinton", ["#lovewins", "@barackobama", "women", "proud"], ["#wakeupamerica", "email", "#benghazi"]
target, pos, neg = "Atheism", ["#freethinker", "#freedom", "religi"], ["jesu", "lord", "bless", "#teamjesus"]
subset_train = assign(target, pos, neg)
pd.crosstab(index=subset_train['labels'], columns=subset_train['Stance'], margins = True)

/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Stance,AGAINST,FAVOR,NONE,All
labels,,,,
-1.0,238,70,113,421
0.0,60,2,1,63
1.0,6,20,3,29
All,304,92,117,513


In [37]:
subset_test = test.query('Target==@target')
test_target = test.query('Target == @target & Stance == @lst').reset_index()
train_target = subset_train.query('Target == @target & Stance == @lst & labels != -1').reset_index()

y_tr = np.array( 2 * train_target.labels - 1, dtype = int)
y_te = np.ones(len(test_target), dtype = np.int8)

# y_tr[np.where(train_target.Stance.values == "AGAINST")] = -1
y_te[np.where(test_target.Stance.values == "AGAINST")] = -1
train_target["y"] = y_tr
test_target["y"] = y_te
    
print(Counter(y_tr), Counter(y_te))


for steps, mk in product(STEPS, mks):
    pipe, _hyperparams = get_model(steps, mk = mk)
    name = "_".join(steps)
    hyperparams = {}
    for hp in _hyperparams:
        hp_name, step = hp
        hyperparams["{}__{}".format(step, hp_name)] = HYPERPARAMS_GRID[hp_name]
        # if step == "greedy-div" and hp_name == "lambdaa":
        #     hyperparams["{}__{}".format(step, hp_name)] = HYPERPARAMS_GRID["lambda2"]
#     print("{}, m={}, params={}".format(name, mk, ";".join(hyperparams.keys()) ))

    clf = GridSearchCV(Pipeline(pipe), hyperparams, 
                cv = CV, n_jobs = N_JOBS, pre_dispatch = N_JOBS, verbose = 0, scoring = scorer, 
                return_train_score = False, error_score = 0.5)
    clf.fit(train_target, train_target.y.values)
    train_score, test_score = clf.score(train_target, train_target.y.values), clf.score(test_target, test_target.y.values)
    val_score = clf.best_score_
    each_f1_scores = each_f1(y_te, clf.predict(test_target) )
#     print("{} => {}[{}]: val:{:.4g}, test:{:.4g} | each_f1:{}".format(target, name, mk, val_score, test_score, each_f1_scores.tolist()))
    S = clf.best_estimator_.steps[-1][1].idxs
    Summaries = train_target.iloc[S].Tweet.values
    Summaries1 = Summaries[:mk].tolist()
    Summaries2 = Summaries[mk:].tolist()
    summ_lengths = [np.mean(train_target.iloc[S].num_tokens.values[:mk]), np.mean(train_target.iloc[S].num_tokens.values[mk:])]
    msg = "{},{},{},{:.4g},{:.4g},{:.2g},{:.2g},{:.4g},{:.4g},{:.4g}".format(
        target, name, mk, val_score, test_score,
        *summ_lengths,
        *each_f1_scores,
        np.mean(each_f1_scores)
        # " ###### ".join(Summaries1),
        # " ###### ".join(Summaries2)
    )
    print(clf.best_estimator_)
    print(msg)
    print()

Counter({-1: 62, 1: 26}) Counter({-1: 160, 1: 32})


/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Jul-22 07:21:56 INFO [MMDGreedy:275]=> fitted MMDGreedy(C=1.0,lambda=0.4,m=5)


Pipeline(steps=[('tok', PandasSelector(tokens)),
                ('idf-bow', BoW(weights='idf')), ('exp', ExpK(gamma=0.0625)),
                ('greedy-diff', MMDGreedy(C=1.0, lambdaa=0.4, mk=5))])
Atheism,tok_idf-bow_exp_greedy-diff,5,0.9181,0.6769,7.6,9.2,0.8738,0.48,0.6769



/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Jul-22 07:21:57 INFO [MMDGreedy:275]=> fitted MMDGreedy(C=0.1,lambda=0.2,m=5)


Pipeline(steps=[('tok', PandasSelector(tokens)),
                ('idf-bow', BoW(weights='idf')), ('exp', ExpK(gamma=0.0625)),
                ('greedy-div',
                 MMDGreedy(C=0.1, diff='div', lambdaa=0.2, mk=5))])
Atheism,tok_idf-bow_exp_greedy-div,5,0.9194,0.6563,8.8,9.4,0.8407,0.4719,0.6563



...

Pipeline(steps=[('tok', PandasSelector(tokens)), ('bow', BoW()),
                ('cos', CosineK()), ('all', AllComp(C=0.1))])
Atheism,tok_bow_cos_all,5,0.5,0.1658,11,9.8,0.04848,0.2831,0.1658

Pipeline(steps=[('tok', PandasSelector(tokens)),
                ('idf-bow', BoW(weights='idf')), ('cos', CosineK()),
                ('all', AllComp(C=0.1))])
Atheism,tok_idf-bow_cos_all,5,0.5,nan,11,9.8,nan,0.2857,nan



/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Jul-22 07:22:01 INFO [AllComp:275]=> fitted AllComp(C=0.1,m=all)


Pipeline(steps=[('tok', PandasSelector(tokens)), ('bow', BoW()),
                ('exp', ExpK(gamma=0.0625)), ('all', AllComp(C=0.1))])
Atheism,tok_bow_exp_all,5,0.5,0.2238,11,9.8,0.1591,0.2885,0.2238



/opt/installed/apps/conda/envs/stancesumm/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Jul-22 07:22:01 INFO [AllComp:275]=> fitted AllComp(C=0.1,m=all)


Pipeline(steps=[('tok', PandasSelector(tokens)),
                ('idf-bow', BoW(weights='idf')), ('exp', ExpK(gamma=0.0625)),
                ('all', AllComp(C=0.1))])
Atheism,tok_idf-bow_exp_all,5,0.5,0.1497,11,9.8,0.01242,0.287,0.1497

